In [1]:
# !pip install -q git+https://github.com/samoturk/mol2vec;

In [2]:
# !pip install -q huggingface-hub transformers catboost xgboost lightgbm torch gensim==3.7.1

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from utils import PreprocessSMILES, SimpleRegressions, SmallNN, LstmRegressor, DataLoader, Training

from utils import load_config, seed_everything

import gensim

print(gensim.__version__) # 3.7.1

2024-04-21 19:22:18.025286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 19:22:20.121548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-21 19:22:20.121905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-04-21 19:22:20.121921: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

3.7.1


## Майнинг данных из SMILES и объединение молекул в смеси

In [4]:
# df = pd.read_csv("materials/test_data_fix_concat.csv")

# Подсчет уникальных значений в столбце "smiles"
# unique_smiles_count = df["smiles"].nunique()

# print("Количество уникальных SMILES:", unique_smiles_count)

In [5]:
# # df = df[df['smiles'].notna()]
# # df = (
# #     df.groupby(["blend_id", "smiles"])
# #     .agg({"oil_property_param_value": "mean"})
# #     .reset_index()
# # )
# from rdkit import Chem
# # df["mol"] = df["smiles"].apply(Chem.MolFromSmiles)
# # df["canonical_smiles"] = df["smiles"].apply(
# #     lambda x: (
# #         Chem.MolToSmiles(
# #             Chem.MolFromSmiles(x), isomericSmiles=True, canonical=True
# #         )
# #         if x is not None
# #         else None
# #     )
# # )
# # rows_to_drop = df[df['mol'].isnull()].index
# # df = df.drop(df[df['blend_id'].isin(df.loc[rows_to_drop, 'blend_id'])].index)
# descriptors = pd.read_csv("materials/descriptors_test.csv")
# # df["descriptors_array"] = (
# #     pd.merge(df, descriptors, on="mol", how="inner")  # ! change
# #     .iloc[:, 5:]
# #     .values.tolist()
# # )

# # df
# descriptors

In [6]:
############################################################################################################################################
directory = './materials/'
preprocessor_test = PreprocessSMILES(directory=directory, base_on='canonical_smiles', task = 'test')
mdf_test = preprocessor_test.preprocess_data("test_data_fix_concat.csv", "descriptors_test.csv")
grouped_mdf_test = preprocessor_test.preprocess_mol2vec(mdf_test)
smiles_df_test = preprocessor_test.smiles2sentence(mdf_test)
joined_df_test = pd.merge(grouped_mdf_test, smiles_df_test, on='blend_id', how='inner').dropna() # TODO: fix nan
joined_df_test = preprocessor_test.embed_smiles(joined_df_test, 'DeepChem/ChemBERTa-10M-MTR') # ibm/MoLFormer-XL-both-10pct DeepChem/ChemBERTa-10M-MTR
joined_df_test.reset_index(drop=True, inplace=True)
mdf_test['graphs'] = preprocessor_test.generate_graphs(mdf_test)
mdf_test.drop(columns=['mol'], inplace=True)
joined_df_test.head(3)
# X, y = preprocessor_test.xy_split(joined_df_test, column = 'embeddings')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Some weights of the model checkpoint at ./materials/embed_model/DeepChem/ChemBERTa-10M-MTR/models--DeepChem--ChemBERTa-10M-MTR/snapshots/b65d0a6af3156071d9519e867d695aa265bb393f were not used when initializing RobertaModel: ['regression.dense.bias', 'regression.dense.weight', 'norm_mean', 'norm_std', 'regression.out_proj.bias', 'regression.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./materials/embed_model/DeepChem/ChemBERTa-10M-MTR/models--DeepChem--ChemBERTa-10M-MTR/snapshots/b65d0

,blend_id,sentence,mol2vec,canonical_smiles,smiles,descriptors_array,similarity_vectors,embeddings
0,0189e1f2-2ab8-11ee-8be1-005056921581,"[2246728737, 3542456614, 2245384272, 117312591...","[-2.5093365, -0.48656785, -1.1924767, -0.49072...","CCCC(C)CCC, CCCCCCC","CCCC(C)CCC, CCCCCCC","[0.0, 0.0, 0.0, 52.0, 16.0, 10.118274536094024...",[0.26666666666666666],"[0.007351607084274292, -0.009765504859387875, ..."
1,02114c6e-eb16-11ed-9974-005056921581,"[2246728737, 3537119515, 2245273601, 84546060,...","[-6.4228168, -1.2596623, -4.4797096, -2.153883...","CCCCOC(=O)C(C)C(=O)OCCCC, CCCCCCCC(C)C(CCCCCC)...","CC(C(OCCCC)=O)C(OCCCC)=O, CCCCCCC(C(C)CCCCCCC)...","[0.0, 0.0, 0.0, 52.0, 16.0, 10.118274536094024...",[0.22857142857142856],"[-0.016469165682792664, 0.0829821452498436, 0...."
2,03ac1c22-6af9-11ed-8d22-005056921581,"[2246728737, 3537123720, 2245277810, 170459982...","[-11.42185, -0.05583578, -8.702778, 0.75527394...","COC(=O)C(C)(C)C, CCCCCC(C)C, CCCC(CC(C)(CC)C(=...","CC(C)(C)C(OC)=O, CC(C)CCCCC, CCC(C(OC)=O)(CC(C...","[0.0, 0.0, 0.0, 130.0, 40.0, 25.29568634023505...","[0.037037037037037035, 0.2894736842105263, 0.0...","[-0.020244603976607323, 0.027134587988257408, ..."


In [7]:
joined_df_test.shape

(138, 8)

## Подготовка данных для обучения на объединенных последовательностях векторов, эмбедингов смеси и близости расстояний между молекулами.


In [8]:
# как similarity_score используется TanimotoSimilarity из rdkit

static_cols = ['embeddings'] #['mol2vec', 'embeddings', 'descriptors_array']
dynamic_cols = [] #['similarity_vectors']

data_loader = DataLoader(joined_df_test, static_cols, dynamic_cols, task = 'test')
dataset, _ = data_loader.get_dataset()

Shape of data_x: (138, 1, 384)


In [9]:
dataset.data_x.shape

(138, 1, 384)

## Тренировка

In [10]:
# Тренировка 3 архитектур нейросеток

config = load_config("config.yml", "embeddings", "torchRegressor")
seed_everything()
trainer = Training(config, dataset.data_x, cross_validation = False)

# pred1 = trainer.average_prediction(test, models)
# pred2 = trainer.weighted_average_prediction(test, models, model_wise=[0.29, 0.33, 0.38], fold_wise=[0.30, 0.35, 0.35])

-----Seed Set!-----


TypeError: __init__() missing 2 required positional arguments: 'y' and 'test_X'

In [ ]:
trainer.trained_models

[GRURegressor(
   (gru): GRU(32, 2, num_layers=2, batch_first=True)
   (linear): Sequential(
     (0): Linear(in_features=26, out_features=1024, bias=True)
     (1): Dropout(p=0.3, inplace=False)
     (2): ReLU()
     (3): Linear(in_features=1024, out_features=512, bias=True)
     (4): Dropout(p=0.3, inplace=False)
     (5): ReLU()
   )
   (head1): Linear(in_features=512, out_features=1, bias=True)
   (loss1): MSELoss()
   (loss3): L1Loss()
 ),
 ConvRegressor(
   (conv_block): Sequential(
     (0): Conv1d(1, 8, kernel_size=(5,), stride=(1,))
     (1): Dropout(p=0.3, inplace=False)
     (2): Conv1d(8, 8, kernel_size=(5,), stride=(1,))
     (3): ReLU()
     (4): Conv1d(8, 16, kernel_size=(5,), stride=(2,))
     (5): Dropout(p=0.3, inplace=False)
     (6): AvgPool1d(kernel_size=(11,), stride=(11,), padding=(0,))
     (7): Conv1d(16, 4, kernel_size=(3,), stride=(3,))
     (8): Flatten(start_dim=1, end_dim=-1)
   )
   (linear): Sequential(
     (0): Linear(in_features=20, out_features=1024,

## Тренировка других моделей

In [ ]:
dataset.data_x.shape

(328, 1, 384)

In [ ]:
config = load_config("config.yml", "embeddings", "SmallNN")
regression_models = SimpleRegressions(X=dataset.data_x.reshape(config['input']), y=dataset.data_y.reshape(config['input'][0]))
regression_models.fit_and_evaluate()

In [ ]:
from utils import SmallNN

nn_model = SmallNN(X=dataset.data_x.reshape(config['input']), y=dataset.data_y.reshape(config['input'][0]), config=config)
nn_model.fit_and_evaluate()

2024-04-21 15:59:30.400555: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-21 15:59:30.401126: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-21 15:59:30.401657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-HPESRLBL): /proc/driver/nvidia/version does not exist
2024-04-21 15:59:30.410955: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/1000
15/15 [==============================] - 3s 53ms/step - loss: 0.7904 - mean_absolute_error: 0.4655 - val_loss: 0.1005 - val_mean_absolute_error: 0.2597 - lr: 0.0300
Epoch 2/1000
15/15 [==============================] - 0s 12ms/step - loss: 0.0652 - mean_absolute_error: 0.2136 - val_loss: 0.0645 - val_mean_absolute_error: 0.2075 - lr: 0.0300
Epoch 3/1000
15/15 [==============================] - 0s 12ms/step - loss: 0.0550 - mean_absolute_error: 0.1814 - val_loss: 0.0631 - val_mean_absolute_error: 0.1963 - lr: 0.0300
Epoch 4/1000
15/15 [==============================] - 0s 11ms/step - loss: 0.0601 - mean_absolute_error: 0.1872 - val_loss: 0.0682 - val_mean_absolute_error: 0.2090 - lr: 0.0300
Epoch 5/1000
15/15 [==============================] - 0s 7ms/step - loss: 0.0576 - mean_absolute_error: 0.1906 - val_loss: 0.0776 - val_mean_absolute_error: 0.2261 - lr: 0.0300
Epoch 6/1000
15/15 [==============================] - 0s 18ms/step - loss: 0.0633 - mean_absolute_error: 0.1963

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
scaler_ = StandardScaler() # RobustScaler

regressor = LstmRegressor(units=50, dropout_rate=0.005, optimizer='rmsprop', epochs=200, 
                          batch_size=8, neurons_1 = 128, neurons_2 = 64, scaler=scaler_)

X_train, X_test, y_train, y_test = train_test_split(
            dataset.data_x.reshape(config['input']), dataset.data_y.reshape(config['input'][0]), test_size=0.2, random_state=42
        )
model, _ = regressor.fit(X_train, y_train, X_test, y_test)

2024-04-21 16:20:24.005823: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-21 16:20:24.008007: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-21 16:20:24.009102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-HPESRLBL): /proc/driver/nvidia/version does not exist
2024-04-21 16:20:24.020101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
20/37 [===============>..............] - ETA: 16s - loss: 1.0267

KeyboardInterrupt: 